In [50]:
import pandas
from torchsummary import summary
import random
import sys 
import os
import numpy as np
import torch
from torch.utils.data import DataLoader, random_split
import torch.optim as optim

In [51]:
# Get the project directory 
current_dir = os.path.abspath('') # Current '\notebooks' directory
project_dir = os.path.abspath(os.path.join(current_dir, '..')) # Move up one level to project root directory

# Add the project directory to sys.path
sys.path.append(project_dir)

# Move up to project directory
os.chdir(project_dir)
os.getcwd()

'/Users/aishwaryaiyer/Documents'

In [52]:
from src.models import *



In [53]:
transformer_model_path = "/Users/aishwaryaiyer/Documents/GitHub/Digital-Asset-Prediction/saved_models/CryptoTransformer_2025-04-09_21-31-23/CryptoTransformer_BEST_R2.pth"
informer_model_path = "/Users/aishwaryaiyer/Documents/GitHub/Digital-Asset-Prediction/saved_models/CryptoInformer_2025-04-10_15-11-19/CryptoInformer_BEST_R2.pth"

In [54]:
transformer_model = CryptoTransformer()  
informer_model = CryptoInformer()


In [55]:
transformer_model.load_state_dict(torch.load(transformer_model_path, map_location=torch.device('cpu')))
informer_model.load_state_dict(torch.load(informer_model_path, map_location=torch.device('cpu')))

transformer_model.eval()
informer_model.eval()

/var/folders/vr/qwxtvrr17fn2qzlm0z4rq4br0000gn/T/ipykernel_70435/2746541776.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  transformer_model.load_state_dict(torch.load(

CryptoInformer(
  (embedding): Linear(in_features=11, out_features=64, bias=True)
  (positional_encoder): SinusoidalPositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (attn): ProbAttention(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder_layers): ModuleList(
    (0-1): 2 x EncoderLayer(
      (attention): AttentionLayer(
        (inner_attention): ProbAttention(
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (query_projection): Linear(in_features=64, out_features=64, bias=True)
        (key_projection): Linear(in_features=64, out_features=64, bias=True)
        (value_projection): Linear(in_features=64, out_features=64, bias=True)
        (out_projection): Linear(in_features=64, out_features=64, bias=True)
      )
      (conv1): Conv1d(64, 256, kernel_size=(1,), stride=(1,))
      (conv2): Conv1d(256, 64, kernel_size=(1,), stride=(1,))
      (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((64,)

In [69]:
CSV_PATH = "data/processed/combined_dataset_v1.csv"
df = pd.read_csv(CSV_PATH, parse_dates=['date'])




df.sort_values(['symbol', 'date'], inplace=True)

# 1. Select a random crypto ticker
all_tickers = df['symbol'].unique()
random_ticker = np.random.choice(all_tickers)
print(f"Selected ticker: {random_ticker}")

# 2. Get the most recent 14 days of data for this ticker
ticker_data = df[df['symbol'] == random_ticker].tail(14)

FileNotFoundError: [Errno 2] No such file or directory: 'data/processed/combined_dataset_v1.csv'

In [ ]:
class CryptoDataset(Dataset):
    def __init__(self, data: pd.DataFrame, features: list, target: str = None, sequence_length: int = 14):
        self.data = data[features].values
        self.target = data[target].values if target else None
        self.sequence_length = sequence_length
        
    def __len__(self):
        # Return 1 since we just want the full sequence for prediction
        return 1 if len(self.data) == self.sequence_length else len(self.data) - self.sequence_length
        
    def __getitem__(self, idx) -> Tuple[torch.Tensor, torch.Tensor]:
        seq = self.data[idx:idx+self.sequence_length]
        if self.target is not None and (idx + self.sequence_length) < len(self.data):
            target = self.target[idx+self.sequence_length]
            return torch.FloatTensor(seq), torch.FloatTensor([target])
        return torch.FloatTensor(seq), torch.FloatTensor([0])  # Dummy target for prediction

In [ ]:
features = ['open', 'high', 'low', 'close', 'volume', 'market_cap', 
            'daily_return', 'sp500', 'treasury_spread', 'fear_greed', 'gold_price_usd']
target = 'close'

# Create a single sequence dataset
dataset = CryptoDataset(ticker_data, features, target)
seq, _ = dataset[0]  # Get our 14-day sequence (shape: [14, num_features])


In [ ]:
# Now make predictions
with torch.no_grad():
    input_seq = seq.unsqueeze(0)  # Add batch dimension
    
    # Apply normalization if you used it during training
    # input_seq = normalizer(input_seq)
    
    transformer_pred = transformer_model(input_seq)
    informer_pred = informer_model(input_seq)

print(f"\nLast actual closing price: {ticker_data['close'].iloc[-1]:.4f}")
print(f"Transformer next day prediction: {transformer_pred.item():.4f}")
print(f"Informer next day prediction: {informer_pred.item():.4f}")


Last actual closing price: 0.0299
Transformer next day prediction: -12.1544
Informer next day prediction: 40.0656
